In [1]:
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import seaborn as sns
import pandas as pd
from tqdm import tqdm
from shapely import wkt
import shapely
import gc
import random

In [2]:
# ADD here path to the datacube
ds = xr.open_zarr("")

In [3]:
#ADD here path to the biomes downloaded from ""
biome = gpd.read_file("")

In [4]:
biome=biome[(biome['BIOME_NUM']== 12.0) & (biome['REALM']=='Palearctic')]

In [5]:
lc_1 = ds['lc_agriculture'].sel(time=slice('2021-01-01', '2021-01-01'))[0]
lc_2 = ds['lc_forest'].sel(time=slice('2021-01-01', '2021-01-01'))[0]
lc_3 = ds['lc_grassland'].sel(time=slice('2021-01-01', '2021-01-01'))[0]
lc_4 = ds['lc_settlement'].sel(time=slice('2021-01-01', '2021-01-01'))[0]
lc_5 = ds['lc_shrubland'].sel(time=slice('2021-01-01', '2021-01-01'))[0]
lc_6 = ds['lc_sparse_vegetation'].sel(time=slice('2021-01-01', '2021-01-01'))[0]
lc_7 = ds['lc_water_bodies'].sel(time=slice('2021-01-01', '2021-01-01'))[0]
lc_8 = ds['lc_wetland'].sel(time=slice('2021-01-01', '2021-01-01'))[0]

In [6]:
lc = np.stack([lc_1, lc_2, lc_3, lc_4, lc_5, lc_6, lc_7, lc_8])
lc = np.argmax(lc, axis=0).reshape(-1)

In [7]:
#ADD here the path to the positive samples
gpd = pd.read_csv("")

In [8]:
gpd = gpd[gpd['time_idx'] == 29]

In [9]:
lc_burned = gpd[['lc_agriculture', 'lc_forest', 'lc_grassland', 'lc_settlement', 'lc_shrubland', 'lc_sparse_vegetation', 'lc_water_bodies', 'lc_wetland']]

In [10]:
lc_burned = np.argmax(lc_burned.to_numpy(), axis=1)

In [11]:
from collections import Counter
clc_counter = Counter(lc)
burned_clc_counter = Counter(lc_burned)
burned_sum = 0
for k, v in clc_counter.items():
    if (
        (burned_clc_counter.get(k) is None) # if never burned
    or (burned_clc_counter.get(k) < 50) # if rarely burned
    or (k in [6,7]) # if cannot be burned
    ):
        burned_clc_counter[k] = 0
    else:
        burned_sum += burned_clc_counter[k]

p = np.array([burned_clc_counter[x]/(clc_counter[x]*burned_sum) for x in list(lc)])

In [12]:
gpd['time'] = pd.to_datetime(gpd['time'])
gpd['YEAR'] = gpd['time'].dt.year

In [13]:
positives_per_year = gpd.groupby('YEAR').count()['time'].to_dict()

In [14]:
from collections import defaultdict
negatives_per_year = {}
negatives_per_day = defaultdict(dict)

for k, v in positives_per_year.items():
    negatives_per_year[k] = positives_per_year[k] * 2

for year in negatives_per_year:
    if year != 2006 and year!=2022:
        negatives_per_day[year]['JF'] = (negatives_per_year[year] // 20)/60
        negatives_per_day[year]['MS'] = (9*negatives_per_year[year] // 10)/240
        negatives_per_day[year]['ND'] = (negatives_per_year[year] // 20)/60
    elif year == 2006:
        negatives_per_day[year]['JF'] = 0
        negatives_per_day[year]['MS'] = (9.5*negatives_per_year[year] // 10)/240
        negatives_per_day[year]['ND'] = (negatives_per_year[year] // 40)/60
    elif year == 2022:
        negatives_per_day[year]['JF'] = (negatives_per_year[year] // 40)/60
        negatives_per_day[year]['MS'] = (9.5*negatives_per_year[year] // 10)/240
        negatives_per_day[year]['ND'] = 0

In [ ]:
#ADDD here the path to save the data
save_dir = ""
dem = ds['dem'].load()
len_x = len(ds['x'])
len_y = len(ds['y'])
s_cl=0
s_seg=0
lag = 30
patch_size = 5
patch_half = 125//2
year = 2005

for t in tqdm(range(lag, len(ds['time']))):
    t_time = pd.to_datetime(ds['time'][t].values) 
    month = pd.DatetimeIndex([ds['time'][t].values]).month[0]
    y = str(year)
    year = pd.DatetimeIndex([ds['time'][t].values]).year[0]

    if y != str(year):
        d_st = {}
        ds_vars = list(ds.keys()) 
        for var in ds_vars:
            if var == 'population' or 'lc' in var:
                if str(year) == '2006':
                    d_st[var] = ds[var].sel(time=slice('2006-04-01', '2006-04-01'))[0].load()
                else:
                    dt = str(year) + '-01-01'
                    d_st[var] = ds[var].sel(time=slice(dt, dt))[0].load()
                        
    x = random.uniform(0,1)
    if month <= 2:
        num_samples, d = divmod(negatives_per_day[year]['JF'], 1)
    elif month >=3 and month <=10:
        num_samples, d = divmod(negatives_per_day[year]['MS'], 1)
    else:
        num_samples, d = divmod(negatives_per_day[year]['ND'], 1)
    if x < d:
        num_samples+=1
    ds_tmp = ds.sel(time=slice(t_time, t_time)).load()
    for i in range(int(num_samples)):
        np_var = {}
        idx = np.random.choice(np.arange(0,len_x*len_y), p=p) 
        y_idx = idx // len_x
        x_idx = idx % len_x
        point = shapely.geometry.Point(ds['dem']['x'][x_idx].values, ds['dem']['y'][y_idx].values)
        while ((x_idx - patch_half < 0) or (x_idx + patch_half + 1 >= len_x) or (y_idx - patch_half< 0) or (y_idx + patch_half + 1 >= len_y)) or \
        (1 in ds_tmp.isel(x=slice(x_idx - patch_half,  x_idx + patch_half + 1), y=slice(y_idx - patch_half, y_idx + patch_half + 1))['burned_areas']) or \
        (True not in point.within(biome.geometry).tolist()) or (pd.isnull(ds_tmp.isel(x=x_idx, y=y_idx)['t2m'])):
            idx = np.random.choice(np.arange(0,len_x*len_y), p=p) 
            y_idx = idx // len_x
            x_idx = idx % len_x
            point = shapely.geometry.Point(ds['dem']['x'][x_idx].values, ds['dem']['y'][y_idx].values)
 
        x = str(dem.isel(x=x_idx)['x'].values)
        y = str(dem.isel(y=y_idx)['y'].values)
       
        ign_date_str  = (t_time).strftime('%Y-%m-%d')
        ign_date_lag_str = (t_time - pd.Timedelta(days=lag-1)).strftime('%Y-%m-%d')

        neg_sample_ds = ds.sel(time=slice(ign_date_lag_str, ign_date_str))

        neg_sample_ds = neg_sample_ds.isel(x=slice(x_idx - patch_half,x_idx + patch_half + 1),
                                  y=slice(y_idx - patch_half,y_idx + patch_half + 1))

        neg_sample_ds_vars = list(neg_sample_ds.keys()) 
        for var in neg_sample_ds_vars:
            if var == 'population' or 'lc' in var:
                del neg_sample_ds[var]
                neg_sample_ds[var] = d_st[var].isel(x=slice(x_idx - patch_half,x_idx + patch_half + 1),
                                          y=slice(y_idx - patch_half,y_idx + patch_half + 1)) 
        
        del neg_sample_ds['spatial_ref']
        neg_sample_ds = neg_sample_ds.load()
      
        neg_sample_ds = neg_sample_ds.isel(x=patch_half, y=patch_half)
        neg_sample_ds['burned_area_has'] = float(0) 
        if s_cl == 0: 
            df = neg_sample_ds.to_dataframe()
            df['time_idx'] = np.arange(0,lag)
            df['sample'] = s_cl
        else:
            df1 = neg_sample_ds.to_dataframe()
            df1['time_idx'] = np.arange(0,lag)
            df1['sample'] = s_cl
            df = pd.concat([df, df1], axis=0)
            del df1
        s_cl+=1
        del neg_sample_ds
        gc.collect()
path_df = save_dir / 'negatives.csv'
df.to_csv(path_df)
                

 19%|█▉        | 1159/5996 [1:23:18<7:47:24,  5.80s/it] 